In [330]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pdfplumber
import re

In [331]:


# Path to your PDF file
pdf_path = "../data/pdf/Dec06.pdf"

with pdfplumber.open(pdf_path, pages = [1]) as pdf:
    # Extract text from each page
    text = ""
    for page in pdf.pages:
        text += page.extract_text();

#matches = re.findall(r'#(.*?)(?=TM)', text)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [294]:
matches = re.findall(r'Southern Oregon \d+..(#[\s\S]+TM Team)', text, re.DOTALL)
matches_date = re.findall(r'Date: \s*(........)', text)
matches

['# Player GS MIN FG 3PT FT ORB-DRB REB PF A TO BLK STL PTS\n20 Morgan Baird * 32 7-13 0-1 4-4 2-3 5 0 2 1 0 0 18\n12 Emma Schmerbach * 29 5-13 0-2 3-8 3-5 8 3 2 0 0 1 13\n1 Meghan McIntyre * 27 2-7 0-1 2-2 1-3 4 5 0 3 0 1 6\n23 Clara Robbins * 17 2-5 0-0 2-2 3-3 6 2 2 1 2 2 6\n10 Bridgette McIntyre * 33 1-3 1-2 0-0 0-4 4 3 1 2 0 1 3\n13 Keeley Wright 12 3-3 0-0 0-2 0-2 2 1 1 1 0 0 6\n32 Eliza DiGiulio 20 2-7 1-4 0-0 2-2 4 2 2 2 1 0 5\n33 Mallory Williams 4 1-3 0-2 2-2 0-0 0 0 0 0 0 0 4\n22 Sierra Scheppele 13 1-2 0-1 0-0 0-1 1 1 0 0 0 0 2\n5 Makayla Scurlock 12 1-1 0-0 0-0 0-6 6 0 0 0 0 0 2\n34 Shakia Teague-Perry 1 0-0 0-0 0-0 0-0 0 0 0 1 0 0 0\nTM Team 0 0-0 0-0 0-0 0-0 0 0 0 0 0 0 0\nTotals - 200 25-57 2-13 13-20 11-29 40 17 10 11 3 5 65\nTeam Summary FG 3PT FT\n1st Quarter 6-15 40.00% 1-2 50.00% 2-2 100.00%\n2nd Quarter 8-15 53.33% 1-4 25.00% 0-0 0.00%\n3rd Quarter 7-15 46.67% 0-4 0.00% 2-6 33.33%\n4th Quarter 4-12 33.33% 0-3 0.00% 9-12 75.00%\nTotal 25-57 43.9% 2-13 15.4% 13-20 6

['# Player GS MIN FG 3PT FT ORB-DRB REB PF A TO BLK STL PTS',
 '12 Emma Schmerbach * 28 7-11 0-1 0-2 4-5 9 2 2 2 1 1 14',
 '20 Morgan Baird * 28 5-11 0-0 4-5 2-2 4 0 3 1 0 0 14',
 '10 Bridgette McIntyre * 30 4-8 4-8 0-0 0-2 2 2 1 2 0 1 12',
 '1 Meghan McIntyre * 32 2-10 0-1 6-10 2-5 7 0 4 2 0 3 10',
 '23 Clara Robbins * 11 0-1 0-0 0-2 0-1 1 4 0 0 1 3 0',
 '32 Eliza DiGiulio 24 5-8 1-2 1-2 1-1 2 1 3 0 1 1 12',
 '33 Mallory Williams 12 2-2 2-2 0-0 0-2 2 1 0 1 0 0 6',
 '5 Makayla Scurlock 12 1-3 0-0 0-0 0-4 4 1 0 1 0 1 2',
 '22 Sierra Scheppele 15 0-1 0-1 0-0 0-1 1 0 1 1 0 0 0',
 '13 Keeley Wright 8 0-1 0-0 0-0 1-0 1 1 1 0 0 0 0',
 'TM Team']

In [328]:
stats = matches[0]
stats

stats = stats.strip().split(sep = '\n')
split_stats = list(map(str.split, stats))
split_stats


def int_convet(x:str):
    try:
        return int(x)
    except ValueError:
        return x
    
def merge_and_pop(lst:list):
    '''merges and pops the name'''
    lst[1] = lst[1] + ' ' + lst[2]
    lst.pop(2)
    return lst
split_stats

def non_start(lst:list):
    '''fills in a value if the player didn't start'''
    if lst[2] != '*':
        lst = lst[:2] + ['-'] + lst[2:]
    return  lst

def get_opponent(raw_text:str):
    """
    The first line should hold something like
    'Southern Oregon (7-0, 1-0) -vs- College of Idaho (2-4, 0-1):'
    """
    line = raw_text.split('\n')[0]
    teams = line.split(' -vs- ')
    team1, team2 = re.findall(r'(^.+)(?: \(\S+\s\S+)', teams[0])[0], re.findall(r'(^.+)(?: \(\S+\s\S+)', teams[1])[0]

    if team1 == 'Southern Oregon':
            return team2
    return team1

opponent = get_opponent(text)
title = split_stats[0] + ['opponent'] + ['date']

player_stats = {key: [] for key in title}

players = split_stats[1:-1]
players = list(map(merge_and_pop, players))
players = list(map(non_start, players))
players

stat_dict = {key: [] for key in title}

for player in players:

    for stat, data in zip(title, player):
        print(stat)
        stat_dict[stat].append(data)
    stat_dict['opponent'].append(opponent)
    stat_dict['date'].append(matches_date)






#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
#
Player
GS
MIN
FG
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
#
Player
GS
MIN
FG
3PT
FT
#
Player
GS
MIN
FG
3PT
FT
ORB-DRB
REB
PF
A
TO
BLK
STL
PTS
opponent
date
#

In [329]:

stat_dict

{'#': ['20',
  '12',
  '1',
  '23',
  '10',
  '13',
  '32',
  '33',
  '22',
  '5',
  '34',
  'TM',
  'Totals',
  'Team',
  '1st',
  '2nd',
  '3rd',
  '4th',
  'Total',
  'Technical',
  'Largest',
  'Lead',
  'College',
  '#',
  '11',
  '34',
  '33',
  '30',
  '4',
  '14',
  '21',
  '3',
  '24'],
 'Player': ['Morgan Baird',
  'Emma Schmerbach',
  'Meghan McIntyre',
  'Clara Robbins',
  'Bridgette McIntyre',
  'Keeley Wright',
  'Eliza DiGiulio',
  'Mallory Williams',
  'Sierra Scheppele',
  'Makayla Scurlock',
  'Shakia Teague-Perry',
  'Team 0',
  '- 200',
  'Summary FG',
  'Quarter 6-15',
  'Quarter 8-15',
  'Quarter 7-15',
  'Quarter 4-12',
  '25-57 43.9%',
  'Fouls: none',
  'Lead: 12',
  'Changed: 2',
  'of Idaho',
  'Player GS',
  'Kylie Waltermeyer',
  'Brooke Beresford',
  'Mia Austin',
  'Taryn Riley',
  'Ashton Adamson',
  'Anna Mooney',
  'McKenzie Hirsch',
  'Lainee Houillon',
  'Brooke Zimmermann'],
 'GS': ['*',
  '*',
  '*',
  '*',
  '*',
  '-',
  '-',
  '-',
  '-',
  '-',

In [292]:
text

'Southern Oregon (7-0, 1-0) -vs- College of Idaho (2-4, 0-1)\n12/06/24 at J.A. Albertson Activity Center\nDate: 12/06/24 Score By Period 1 2 3 4 Total\nTime: 5:30 PM\nSouthern Oregon 15 17 16 17 65\nAttendance: 212\nSite: J.A. Albertson Activity Center College of Idaho 15 11 17 12 55\nReferees: Sheldon Marz, Kenny Thomas, Sam Lungen\nSouthern Oregon 65\n# Player GS MIN FG 3PT FT ORB-DRB REB PF A TO BLK STL PTS\n20 Morgan Baird * 32 7-13 0-1 4-4 2-3 5 0 2 1 0 0 18\n12 Emma Schmerbach * 29 5-13 0-2 3-8 3-5 8 3 2 0 0 1 13\n1 Meghan McIntyre * 27 2-7 0-1 2-2 1-3 4 5 0 3 0 1 6\n23 Clara Robbins * 17 2-5 0-0 2-2 3-3 6 2 2 1 2 2 6\n10 Bridgette McIntyre * 33 1-3 1-2 0-0 0-4 4 3 1 2 0 1 3\n13 Keeley Wright 12 3-3 0-0 0-2 0-2 2 1 1 1 0 0 6\n32 Eliza DiGiulio 20 2-7 1-4 0-0 2-2 4 2 2 2 1 0 5\n33 Mallory Williams 4 1-3 0-2 2-2 0-0 0 0 0 0 0 0 4\n22 Sierra Scheppele 13 1-2 0-1 0-0 0-1 1 1 0 0 0 0 2\n5 Makayla Scurlock 12 1-1 0-0 0-0 0-6 6 0 0 0 0 0 2\n34 Shakia Teague-Perry 1 0-0 0-0 0-0 0-0 0 0 0

In [288]:
stat_dict

{'#': ['12',
  'Emma Schmerbach',
  '20',
  'Morgan Baird',
  '10',
  'Bridgette McIntyre',
  '1',
  'Meghan McIntyre',
  '23',
  'Clara Robbins',
  '32',
  'Eliza DiGiulio',
  '33',
  'Mallory Williams',
  '5',
  'Makayla Scurlock',
  '22',
  'Sierra Scheppele',
  '13',
  'Keeley Wright'],
 'Player': ['*', '*', '*', '*', '*', '-', '-', '-', '-', '-'],
 'GS': ['28', '28', '30', '32', '11', '24', '12', '12', '15', '8'],
 'MIN': ['7-11',
  '5-11',
  '4-8',
  '2-10',
  '0-1',
  '5-8',
  '2-2',
  '1-3',
  '0-1',
  '0-1'],
 'FG': ['0-1', '0-0', '4-8', '0-1', '0-0', '1-2', '2-2', '0-0', '0-1', '0-0'],
 '3PT': ['0-2',
  '4-5',
  '0-0',
  '6-10',
  '0-2',
  '1-2',
  '0-0',
  '0-0',
  '0-0',
  '0-0'],
 'FT': ['4-5', '2-2', '0-2', '2-5', '0-1', '1-1', '0-2', '0-4', '0-1', '1-0'],
 'ORB-DRB': ['9', '4', '2', '7', '1', '2', '2', '4', '1', '1'],
 'REB': ['2', '0', '2', '0', '4', '1', '1', '1', '0', '1'],
 'PF': ['2', '3', '1', '4', '0', '3', '0', '0', '1', '1'],
 'A': ['2', '1', '2', '2', '0', '0',

In [334]:
text

'Southern Oregon (7-0, 1-0) -vs- College of Idaho (2-4, 0-1)\n12/06/24 at J.A. Albertson Activity Center\nDate: 12/06/24 Score By Period 1 2 3 4 Total\nTime: 5:30 PM\nSouthern Oregon 15 17 16 17 65\nAttendance: 212\nSite: J.A. Albertson Activity Center College of Idaho 15 11 17 12 55\nReferees: Sheldon Marz, Kenny Thomas, Sam Lungen\nSouthern Oregon 65\n# Player GS MIN FG 3PT FT ORB-DRB REB PF A TO BLK STL PTS\n20 Morgan Baird * 32 7-13 0-1 4-4 2-3 5 0 2 1 0 0 18\n12 Emma Schmerbach * 29 5-13 0-2 3-8 3-5 8 3 2 0 0 1 13\n1 Meghan McIntyre * 27 2-7 0-1 2-2 1-3 4 5 0 3 0 1 6\n23 Clara Robbins * 17 2-5 0-0 2-2 3-3 6 2 2 1 2 2 6\n10 Bridgette McIntyre * 33 1-3 1-2 0-0 0-4 4 3 1 2 0 1 3\n13 Keeley Wright 12 3-3 0-0 0-2 0-2 2 1 1 1 0 0 6\n32 Eliza DiGiulio 20 2-7 1-4 0-0 2-2 4 2 2 2 1 0 5\n33 Mallory Williams 4 1-3 0-2 2-2 0-0 0 0 0 0 0 0 4\n22 Sierra Scheppele 13 1-2 0-1 0-0 0-1 1 1 0 0 0 0 2\n5 Makayla Scurlock 12 1-1 0-0 0-0 0-6 6 0 0 0 0 0 2\n34 Shakia Teague-Perry 1 0-0 0-0 0-0 0-0 0 0 0